# Cargar datos y librerias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Basic
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from statistics import mode
import os
import glob
import random
import cv2

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle

import cv2
# Features 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from skimage import feature
# Metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score #c alculate accuracy
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
# Classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import svm
#Save model
import joblib  

# Carga de datos 1

In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/2-East Asian/train"

list_people = os.listdir(path)

type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

train_data = np.zeros((cont,168*192*2))
train_data = train_data.astype('float32')

y_train = np.zeros(cont)
y_train = y_train.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        image_merge = image_merge.ravel()
        train_data[cont2] += image_merge 

        y_train[cont2] = i[-2] + i[-1]

        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  2669


In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/2-East Asian/test"

list_people = os.listdir(path)

subcadena = "yaleB" #la subcadena que queremos localizar
type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

test_data = np.zeros((cont,168*192*2))
test_data = test_data.astype('float32')

y_test = np.zeros(cont)
y_test = y_test.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        image_merge = image_merge.ravel()

        test_data[cont2] += image_merge 

        y_test[cont2] = i[-2] + i[-1]
        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  1000


#Test

In [ ]:
#image = cv2.imread("/content/drive/MyDrive/Colab Notebooks/Base-De-Datos/datasetVGG/1-African American/train/african_001/0001_02.jpg", cv2.IMREAD_COLOR)

In [ ]:
#image = cv2.resize(image, (168,192))
#H,S,V = cv2.split(image)
#S.shape

In [ ]:
#image_merge = np.zeros((192,168,2))

#image_merge[:,:,0] = S
#image_merge[:,:,1] = V

In [ ]:
#print(V)

In [ ]:
#print(image_merge[:,:,1])

In [ ]:
#train_data, y_train = extract_data(path_train,"*.jpg",True)

In [ ]:
#test_data, y_test = extract_data(path_test,"*.jpg",True)

In [ ]:
#import sys
#np.set_printoptions(threshold=sys.maxsize)
#y_train

# Separar datos en train y test

In [ ]:
#nsamples, nx, ny, chanels = train_data.shape

#train_data = train_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(train_data)

train_data, X_sparse, y_train = shuffle(train_data, X_sparse,y_train, random_state=0)

#train_data = train_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
#nsamples, nx, ny, chanels = test_data.shape

#test_data = test_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(test_data)

test_data, X_sparse, y_test = shuffle(test_data, X_sparse,y_test, random_state=0)

#test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data = train_data.astype("uint8")
#nsamples1, nx1, ny1, chanels = train_data.shape
#train_data = train_data.reshape((nsamples1,nx1*ny1*chanels))
train_data = train_data/255

test_data = test_data.astype("uint8")
#nsamples, nx, ny, chanels = test_data.shape
#test_data = test_data.reshape((nsamples,nx*ny*chanels))
test_data = test_data/255

# PCA 

## Extracción de caracteristicas 

In [ ]:
n_components = 150

print("Extrayendo %d eigenfaces de %d caras"
      % (n_components, train_data.shape[0]))

pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(train_data)


#eigenfaces = pca.components_.reshape((n_components,192, 162))

print("Projecting the input data on the eigenfaces orthonormal basis")

train_data_pca = pca.transform(train_data)
test_data_pca = pca.transform(test_data)

Extrayendo 150 eigenfaces de 2669 caras
Projecting the input data on the eigenfaces orthonormal basis


texto en negrita## SVM


## SVM

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

clf_pca_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_pca_svm = clf_pca_svm.fit(train_data_pca,y_train )

print("Best estimator found by grid search:")
print(clf_pca_svm.best_estimator_)

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.005)


In [ ]:
y_pred = clf_pca_svm.predict(test_data_pca)
report = classification_report(y_test, y_pred,digits=4,output_dict=True)
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.8431    0.8600    0.8515        50
           2     0.7547    0.8000    0.7767        50
           3     0.6154    0.6400    0.6275        50
           4     0.5273    0.5800    0.5524        50
           5     0.5667    0.6800    0.6182        50
           6     0.4921    0.6200    0.5487        50
           7     0.9792    0.9400    0.9592        50
           8     0.7347    0.7200    0.7273        50
           9     0.7317    0.6000    0.6593        50
          10     0.9783    0.9000    0.9375        50
          11     0.5745    0.5400    0.5567        50
          12     0.5600    0.5600    0.5600        50
          13     0.8148    0.8800    0.8462        50
          14     0.6111    0.4400    0.5116        50
          15     0.5769    0.6000    0.5882        50
          16     0.5610    0.4600    0.5055        50
          17     0.7547    0.8000    0.7767        50
          18     0.6250    

In [ ]:
import pandas
df = pandas.DataFrame(report).transpose()
df.to_excel("svmPCA.xlsx")  

In [ ]:
def specificity(y_true,y_pred):
        cm=confusion_matrix(y_true, y_pred)
        FP = cm.sum(axis=0) - np.diag(cm)  
        FN = cm.sum(axis=1) - np.diag(cm)
        TP = np.diag(cm)
        TN = cm.sum() - (FP + FN + TP)
        Specificity = TN/(TN+FP)    
        return np.mean(Specificity)
        
print(specificity(y_test, y_pred))

0.9832105263157895


## knn

In [ ]:
clf_PCA_Knn_3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf_lbp_Kn_3 = clf_PCA_Knn_3.fit(train_data_pca,y_train)

In [ ]:
y_pred = clf_PCA_Knn_3.predict(test_data_pca)
report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()                   
df.to_excel("knnPCA.xlsx")
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.8095    0.3400    0.4789        50
           2     0.2807    0.6400    0.3902        50
           3     0.2785    0.4400    0.3411        50
           4     0.2353    0.3200    0.2712        50
           5     0.1961    0.2000    0.1980        50
           6     0.2453    0.2600    0.2524        50
           7     0.6176    0.4200    0.5000        50
           8     0.3662    0.5200    0.4298        50
           9     0.1518    0.5800    0.2407        50
          10     0.6875    0.6600    0.6735        50
          11     0.1765    0.0600    0.0896        50
          12     0.4815    0.2600    0.3377        50
          13     0.8824    0.3000    0.4478        50
          14     0.4000    0.0400    0.0727        50
          15     0.5000    0.2200    0.3056        50
          16     0.7500    0.1200    0.2069        50
          17     0.7647    0.2600    0.3881        50
          18     0.6000    

In [ ]:
from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(y_test, y_pred))

0.2871358920151689


In [ ]:
print(specificity(y_test, y_pred))    

0.9640000000000002


## Gauss

In [ ]:
clf_PCA_Gauss = GaussianNB()

In [ ]:
clf_lbp_Gauss = clf_PCA_Gauss.fit(train_data_pca,y_train)

In [ ]:
y_pred = clf_lbp_Gauss.predict(test_data_pca)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussPCA.xlsx")
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.7619    0.6400    0.6957        50
           2     0.6667    0.6000    0.6316        50
           3     0.5161    0.3200    0.3951        50
           4     0.2458    0.5800    0.3452        50
           5     0.7143    0.5000    0.5882        50
           6     0.4333    0.5200    0.4727        50
           7     0.8958    0.8600    0.8776        50
           8     0.5000    0.5800    0.5370        50
           9     0.5500    0.4400    0.4889        50
          10     0.9778    0.8800    0.9263        50
          11     0.3774    0.4000    0.3883        50
          12     0.4286    0.4200    0.4242        50
          13     0.7333    0.6600    0.6947        50
          14     0.3929    0.2200    0.2821        50
          15     0.6170    0.5800    0.5979        50
          16     0.3846    0.4000    0.3922        50
          17     0.6607    0.7400    0.6981        50
          18     0.5652    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.5322801265756952


In [ ]:
print(specificity(y_test, y_pred)) 

0.9765263157894738


# LDA

## Extracción de caracteristicas 

In [ ]:
lda = LDA(n_components=19)
X_train = lda.fit_transform(train_data, y_train)
train_data_LDA = lda.transform(train_data)
test_data_LDA = lda.transform(test_data)

## SVM

In [ ]:
clf_lda_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_lda_svm = clf_lda_svm.fit(train_data_LDA,y_train )

print("Best estimator found by grid search:")
print(clf_lda_svm.best_estimator_)

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.01)


In [ ]:
y_pred_lda_svm = clf_lda_svm.predict(test_data_LDA)
report = classification_report(y_test, y_pred_lda_svm,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("svmLDA.xlsx")  
print(classification_report(y_test, y_pred_lda_svm,digits=4))

              precision    recall  f1-score   support

           1     1.0000    0.2400    0.3871        50
           2     0.5763    0.6800    0.6239        50
           3     0.2593    0.2800    0.2692        50
           4     0.4407    0.5200    0.4771        50
           5     0.5500    0.4400    0.4889        50
           6     0.3506    0.5400    0.4252        50
           7     0.9412    0.6400    0.7619        50
           8     0.4533    0.6800    0.5440        50
           9     0.3621    0.4200    0.3889        50
          10     0.7857    0.8800    0.8302        50
          11     0.3585    0.3800    0.3689        50
          12     0.4630    0.5000    0.4808        50
          13     0.6735    0.6600    0.6667        50
          14     0.5333    0.3200    0.4000        50
          15     0.5294    0.3600    0.4286        50
          16     0.3889    0.1400    0.2059        50
          17     0.6066    0.7400    0.6667        50
          18     0.6562    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lda_svm))

0.4732276553597535


In [ ]:
print(specificity(y_test, y_pred_lda_svm)) 

0.9735789473684211


## knn

In [ ]:
clf_LDA_Knn_3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf_LDA_Knn_3 = clf_PCA_Knn_3.fit(train_data_LDA,y_train)

In [ ]:
y_pred = clf_LDA_Knn_3.predict(test_data_LDA)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("knnLDA.xlsx")  
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.8378    0.6200    0.7126        50
           2     0.5211    0.7400    0.6116        50
           3     0.3704    0.4000    0.3846        50
           4     0.3933    0.7000    0.5036        50
           5     0.4789    0.6800    0.5620        50
           6     0.4762    0.6000    0.5310        50
           7     0.8627    0.8800    0.8713        50
           8     0.7442    0.6400    0.6882        50
           9     0.3913    0.3600    0.3750        50
          10     0.8723    0.8200    0.8454        50
          11     0.4565    0.4200    0.4375        50
          12     0.5400    0.5400    0.5400        50
          13     0.7255    0.7400    0.7327        50
          14     0.6071    0.3400    0.4359        50
          15     0.5526    0.4200    0.4773        50
          16     0.5185    0.2800    0.3636        50
          17     0.6071    0.6800    0.6415        50
          18     0.6744    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.5538853619433354


In [ ]:
print(specificity(y_test, y_pred)) 

0.9776315789473685


## Gauss

In [ ]:
clf_LDA_Gauss = GaussianNB()

In [ ]:
clf_LDA_Gauss = clf_LDA_Gauss.fit(train_data_LDA,y_train)

In [ ]:
y_pred = clf_LDA_Gauss.predict(test_data_LDA)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussLDA.xlsx") 
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     1.0000    0.0600    0.1132        50
           2     0.7955    0.7000    0.7447        50
           3     0.5000    0.3000    0.3750        50
           4     0.5333    0.6400    0.5818        50
           5     0.8846    0.4600    0.6053        50
           6     0.3696    0.6800    0.4789        50
           7     1.0000    0.4000    0.5714        50
           8     0.3478    0.8000    0.4848        50
           9     0.3404    0.3200    0.3299        50
          10     0.8571    0.7200    0.7826        50
          11     0.3182    0.5600    0.4058        50
          12     0.4203    0.5800    0.4874        50
          13     0.8919    0.6600    0.7586        50
          14     0.7857    0.2200    0.3437        50
          15     0.6818    0.3000    0.4167        50
          16     0.5556    0.1000    0.1695        50
          17     0.6154    0.6400    0.6275        50
          18     0.7368    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.4742548230167095


In [ ]:
print(specificity(y_test, y_pred)) 

0.9734210526315789


# Carga de datos 2

In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/2-East Asian/train"

list_people = os.listdir(path)

type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

train_data = np.zeros((cont,192,168,2))
train_data = train_data.astype('float32')

y_train = np.zeros(cont)
y_train = y_train.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        train_data[cont2] += image_merge 

        y_train[cont2] = i[-2] + i[-1]

        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  2669


In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/2-East Asian/test"

list_people = os.listdir(path)

subcadena = "yaleB" #la subcadena que queremos localizar
type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

test_data = np.zeros((cont,192,168,2))
test_data = test_data.astype('float32')

y_test = np.zeros(cont)
y_test = y_test.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        

        test_data[cont2] += image_merge 

        y_test[cont2] = i[-2] + i[-1]
        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  1000


# Separar datos en train y test

In [ ]:
nsamples, nx, ny, chanels = train_data.shape

train_data = train_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(train_data)

train_data, X_sparse, y_train = shuffle(train_data, X_sparse,y_train, random_state=0)

train_data = train_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
nsamples, nx, ny, chanels = test_data.shape

test_data = test_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(test_data)

test_data, X_sparse, y_test = shuffle(test_data, X_sparse,y_test, random_state=0)

test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data = train_data.astype("uint8")
nsamples1, nx1, ny1, chanels = train_data.shape
train_data = train_data.reshape((nsamples1,nx1*ny1*chanels))
train_data = train_data/255

test_data = test_data.astype("uint8")
nsamples, nx, ny, chanels = test_data.shape
test_data = test_data.reshape((nsamples,nx*ny*chanels))
test_data = test_data/255

In [ ]:
train_data = train_data.reshape((nsamples1,nx1,ny1,chanels))
test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data_S = np.zeros((nsamples1,192,168))
train_data_S = train_data[:,:,:,0]
train_data_V = np.zeros((nsamples1,192,168))
train_data_V = train_data[:,:,:,1]

test_data_S = np.zeros((nsamples,192,168))
test_data_S = test_data[:,:,:,0]
test_data_V = np.zeros((nsamples1,192,168))
test_data_V = test_data[:,:,:,1]

# LBP


## Extracción de caracteristicas

In [ ]:
# https://github.com/AsoStrife/Computer-Vision-Project

def blockshaped(arr, nrows, ncols):

    h, w = arr.shape
    return (arr.reshape(h//nrows, nrows, -1, ncols)
            .swapaxes(1,2)
            .reshape(-1, nrows, ncols))
def getHistogram(imgArray):
    hist, bin_edges = np.histogram(imgArray, density=True)
    return hist

In [ ]:
# tamaño de train y test
cont_train = train_data.shape[0]
cont_test = test_data.shape[0]
print(cont_test)

1000


In [ ]:
#train_data_LBP = train_data.reshape((cont_train,nx,ny))
lbp_feactures  = np.zeros((cont_train,nx,ny))

eps = 1e-7
radius = 1
n_points = 8 * radius

lbp_hist_S = np.zeros((cont_train,1440))

for i in range(0,cont_train): 
    lbp = feature.local_binary_pattern(train_data_S[i], n_points, radius)
    lbp_feactures[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_S[i] += x[0]

In [ ]:
#train_data_LBP = train_data.reshape((cont_train,nx,ny))
lbp_feactures  = np.zeros((cont_train,nx,ny))

eps = 1e-7
radius = 1
n_points = 8 * radius

lbp_hist_V = np.zeros((cont_train,1440))

for i in range(0,cont_train): 
    lbp = feature.local_binary_pattern(train_data_V[i], n_points, radius)
    lbp_feactures[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_V[i] += x[0]

In [ ]:
#test_data_LBP = test_data.reshape((cont_test,nx,ny))
lbp_feactures_test  = np.zeros((cont_test,nx,ny))


lbp_hist_test_S = np.zeros((cont_test,1440))

for i in range(0,cont_test): 
    lbp = feature.local_binary_pattern(test_data_S[i], n_points, radius)
    lbp_feactures_test[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures_test[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_test_S[i] += x[0]
    

In [ ]:
#test_data_LBP = test_data.reshape((cont_test,nx,ny))
lbp_feactures_test  = np.zeros((cont_test,nx,ny))


lbp_hist_test_V = np.zeros((cont_test,1440))

for i in range(0,cont_test): 
    lbp = feature.local_binary_pattern(test_data_V[i], n_points, radius)
    lbp_feactures_test[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures_test[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_test_V[i] += x[0]
    

In [ ]:
lbp_hist_test_V.shape

(1000, 1440)

In [ ]:
lbp_hist = np.concatenate((lbp_hist_S,lbp_hist_V),axis=1)
lbp_hist.shape

(2669, 2880)

In [ ]:
lbp_hist_test = np.concatenate((lbp_hist_test_S,lbp_hist_test_V),axis=1)
lbp_hist_test.shape

(1000, 2880)

## SMV

In [ ]:
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf_lbp_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_lbp_svm = clf_lbp_svm.fit(lbp_hist,y_train )

print("El mejor conjunto de caracteriticas es: ")
print(clf_lbp_svm.best_estimator_)

El mejor conjunto de caracteriticas es: 
SVC(C=100000.0, class_weight='balanced', gamma=0.001)


In [ ]:
y_pred_lbp_svm = clf_lbp_svm.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_svm,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("svmLBP.xlsx")
print(classification_report(y_test, y_pred_lbp_svm,digits=4))

              precision    recall  f1-score   support

           1     0.8148    0.8800    0.8462        50
           2     0.7069    0.8200    0.7593        50
           3     0.5690    0.6600    0.6111        50
           4     0.6207    0.7200    0.6667        50
           5     0.6833    0.8200    0.7455        50
           6     0.7895    0.6000    0.6818        50
           7     0.8393    0.9400    0.8868        50
           8     0.7917    0.7600    0.7755        50
           9     0.6286    0.4400    0.5176        50
          10     0.9792    0.9400    0.9592        50
          11     0.6596    0.6200    0.6392        50
          12     0.5345    0.6200    0.5741        50
          13     0.7818    0.8600    0.8190        50
          14     0.6923    0.5400    0.6067        50
          15     0.7222    0.7800    0.7500        50
          16     0.6739    0.6200    0.6458        50
          17     0.8125    0.7800    0.7959        50
          18     0.7000    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_svm))

0.7119755157572578


In [ ]:
print(specificity(y_test, y_pred_lbp_svm)) 

0.985578947368421


## KNN

In [ ]:
clf_lbp_Kn_3 = KNeighborsClassifier(n_neighbors=7)

In [ ]:
clf_lbp_Kn_3 = clf_lbp_Kn_3.fit(lbp_hist,y_train)

In [ ]:
y_pred_lbp_knn = clf_lbp_Kn_3.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_knn,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()                   
df.to_excel("knnLBP.xlsx")  
print(classification_report(y_test, y_pred_lbp_knn,digits=4))

              precision    recall  f1-score   support

           1     0.4516    0.8400    0.5874        50
           2     0.5000    0.5000    0.5000        50
           3     0.3548    0.2200    0.2716        50
           4     0.3030    0.4000    0.3448        50
           5     0.2982    0.3400    0.3178        50
           6     0.3636    0.1600    0.2222        50
           7     0.2080    0.9400    0.3406        50
           8     0.4118    0.1400    0.2090        50
           9     0.4333    0.2600    0.3250        50
          10     0.5493    0.7800    0.6446        50
          11     0.1176    0.0400    0.0597        50
          12     0.7500    0.1800    0.2903        50
          13     0.6957    0.3200    0.4384        50
          14     0.6250    0.1000    0.1724        50
          15     0.3944    0.5600    0.4628        50
          16     0.3514    0.2600    0.2989        50
          17     0.5833    0.4200    0.4884        50
          18     0.3659    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_knn))

0.356915668242514


In [ ]:
print(specificity(y_test, y_pred_lbp_knn))  

0.967421052631579


## Gauss

In [ ]:
clf_lbp_Gauss = GaussianNB()

In [ ]:
clf_lbp_Gauss = clf_lbp_Gauss.fit(lbp_hist,y_train)

In [ ]:
y_pred_lbp_gaus = clf_lbp_Gauss.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_gaus,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussLBP.xlsx")
print(classification_report(y_test, y_pred_lbp_gaus,digits=4))

              precision    recall  f1-score   support

           1     0.5714    0.0800    0.1404        50
           2     0.4250    0.3400    0.3778        50
           3     0.1220    0.1000    0.1099        50
           4     0.0714    0.0200    0.0312        50
           5     0.2892    0.4800    0.3609        50
           6     0.2895    0.2200    0.2500        50
           7     0.3611    0.7800    0.4937        50
           8     0.4762    0.4000    0.4348        50
           9     0.1250    0.0200    0.0345        50
          10     0.4023    0.7000    0.5109        50
          11     0.2143    0.2400    0.2264        50
          12     0.2344    0.3000    0.2632        50
          13     0.4167    0.5000    0.4545        50
          14     0.4062    0.2600    0.3171        50
          15     0.3077    0.1600    0.2105        50
          16     0.5000    0.0800    0.1379        50
          17     0.3494    0.5800    0.4361        50
          18     0.3696    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_gaus))

0.29955190567413015


In [ ]:
print(specificity(y_test, y_pred_lbp_gaus)) 

0.964842105263158
